In [1]:
 # !pip install mediapipe opencv-python

In [2]:
import mediapipe as mp
import cv2 
import numpy as np
import uuid
import os

In [3]:
mp_drawing = mp.solutions.drawing_utils

In [4]:
mp_hands = mp.solutions.hands

In [8]:
cap = cv2.VideoCapture(0)

# Set up frame processing counter
frame_count = 0
process_every_n_frame = 5  # Process every 5th frame

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize frame to reduce processing load
        frame = cv2.resize(frame, (640, 480))

        # Convert BGR to RGB for MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        image = cv2.flip(image, 1)

        # Process only every nth frame to reduce load
        if frame_count % process_every_n_frame == 0:
            results = hands.process(image)
            frame_count = 0  # Reset frame counter after processing

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Optional: Draw hand landmarks if detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the processed frame
        cv2.imshow('Hand Tracking', image)
        
        frame_count += 1  # Increment frame counter
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the webcam and destroy all windows
cap.release()
cv2.destroyAllWindows()

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
# Start hand detection and gesture recognition
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert BGR to RGB for MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detect hand landmarks
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Recognize gestures
        gesture = "Unknown"
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract landmark positions
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

                # Gesture: Thumbs Up
                if thumb_tip.y < index_finger_tip.y:
                    gesture = "Thumbs Up"
                
                # Gesture: Open Hand (all fingers extended)
                elif (index_finger_tip.y < middle_finger_tip.y < ring_finger_tip.y < pinky_tip.y):
                    gesture = "Open Hand"
                
                # Gesture: Fist (all fingers curled)
                elif (index_finger_tip.y > middle_finger_tip.y > ring_finger_tip.y > pinky_tip.y and
                      thumb_tip.y > index_finger_tip.y):
                    gesture = "Fist"
                
                # Gesture: Victory (Peace Sign - index and middle fingers up)
                elif (index_finger_tip.y < thumb_tip.y and middle_finger_tip.y < thumb_tip.y and
                      ring_finger_tip.y > middle_finger_tip.y and pinky_tip.y > ring_finger_tip.y):
                    gesture = "Victory"
                
                # Gesture: Pointing (Index Finger Up)
                elif (index_finger_tip.y < thumb_tip.y and 
                      middle_finger_tip.y > index_finger_tip.y and 
                      ring_finger_tip.y > middle_finger_tip.y and 
                      pinky_tip.y > ring_finger_tip.y):
                    gesture = "Pointing"

        # Display detected gesture
        cv2.putText(image, f'Gesture: {gesture}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the output
        cv2.imshow('Hand Gesture Recognition', image)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
